In [3]:
from binaryninja.architecture import Architecture
from binaryninja.platform import Platform
from binaryninja.typelibrary import TypeLibrary
from binaryninja.enums import StructureVariant, NamedTypeReferenceClass
from binaryninja.types import (Type, Tuple, PointerType, StructureType, EnumerationType, FunctionType, EnumerationMember, EnumerationBuilder, QualifiedName, FunctionParameter, PointerBuilder)
from binaryninja.log import log_to_stdout
from binaryninja.enums import LogLevel
from functools import cache
from typing import Optional, Set, List, DefaultDict
from pathlib import Path
import json, codecs
from collections import defaultdict


log_to_stdout(LogLevel.WarningLog)
api_namespaces = {}
for file in Path("win32json/api/").glob("*.json"):
    with codecs.open(str(file), "r", "utf-8-sig") as f:
        api_namespaces[file.stem] = json.load(f)
print("Success")

Success


## Create Ordinal/GUID Mapping

## Import the original JSON files

In [4]:
from tqdm.notebook import tqdm

mapping = {}
items = list(Path("/Users/peterlafosse/src/binaryninja/typelib/x86_64/").glob("*.dll.bntl"))
for file in tqdm(items, total=len(items), unit="typelibs", desc="building ordinal mapping file"):
    filename = file.name
    file = str(file)
    tl = TypeLibrary.load_from_file(file)
    if tl is None:
        print(file)
    assert tl is not None, f"Failed to open {file}"
    ordinals_name = tl.query_metadata("ordinals")
    ordinals = tl.query_metadata(str(ordinals_name))
    mapping[filename] = {}
    mapping[filename]["guid"] = tl.guid
    mapping[filename]["ordinals"] = ordinals_name
    mapping[filename][ordinals_name] = ordinals
with open("mappingfile.json", "w") as f:
    json.dump(mapping, f)
print("Success")

building ordinal mapping file:   0%|          | 0/30 [00:00<?, ?typelibs/s]

Success


## Remap the functions by DllImport

In [5]:

libs = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None))))
types = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None)))
arch_list = ["Arm64", "X64", "X86"]
def assign(archs:List, api:str, type_name:str, type):
    global types
    global libs
    if len(archs) > 0:
        for arch in type["Architectures"]:
            types[arch][api][type_name] = type
    else:
        for arch in arch_list:
            types[arch][api][type_name] = type

def composite_type(type, prefix):
    for nested_type in type["NestedTypes"]:
        new_prefix = f"{prefix}{nested_type['Name']}"
        assign(nested_type["Architectures"], name, new_prefix, nested_type)
        composite_type(nested_type, new_prefix + "::")

for name, obj in api_namespaces.items():
    for func in obj["Functions"]:
        dll = func["DllImport"].lower()
        if len(func["Architectures"]) > 0:
            for arch in func["Architectures"]:
                libs[arch][dll][name][func["Name"]] = func
        else:
            for arch in arch_list:
                libs[arch][dll][name][func["Name"]] = func

    for type in obj["Types"]:
        assign(type["Architectures"], name, type["Name"], type)
        if type["Kind"] in ("Struct", "Union"):
            composite_type(type, f"{type['Name']}::")

# Spot check expected types
assert "APPLICATION_RECOVERY_CALLBACK" in types['X86']['System.WindowsProgramming']
assert "PROCESS_HEAP_ENTRY" in types["X64"]["System.Memory"]
assert "PROCESS_HEAP_ENTRY::_Anonymous_e__Union" in types["X64"]["System.Memory"]
assert "PROCESS_HEAP_ENTRY::_Anonymous_e__Union::_Block_e__Struct" in types["X64"]["System.Memory"]
assert "PROCESS_HEAP_ENTRY::_Anonymous_e__Union::_Block_e__Struct::HANDLE" not in types["X64"]["System.Memory"]
assert "HANDLE" in types["X64"]["Foundation"]

# Verify that each type definition is reachable
def verify(api, arch, t, name):
    kind = t["Kind"]
    if kind in ("PointerTo", "Array", "LPArray"):
        verify(api, arch, t["Child"], name)
    elif kind == "ApiRef":
        try:
            types[arch][t["Api"]][t["Name"]]
        except KeyError:
            types[arch][t["Api"]][f"{name}::{t['Name']}"]
    elif kind in ("Struct", "Union"):
        for type in t["NestedTypes"]:
            verify(api, arch, type, f"{name}::{t['Name']}" if name is None else name)

for arch, apis in types.items():
    for api, type_dict in apis.items():
        for name, t in type_dict.items():
            verify(api, arch, t, name)
print("Success")

Success


## Create a reverse mapping of api,type_name
So we can figure out which are the common types

In [2]:
def deps(types, t, prefix, seen, cur_api, nested=False) -> Set[Tuple[str, str]]:
    assert isinstance(t, dict)
    kind = t["Kind"]
    if kind == "Native":
        return set()
    elif t["Kind"] == "ApiRef":
        item = (t['Api'], prefix + t["Name"])
        result = set([item])
        if item not in seen:
            seen.add(item)
            if nested:
                result |= deps(types, [t['Api']][prefix + t["Name"]], prefix, seen, t['Api'])
        return result
    elif t["Kind"] == "NativeTypedef":
        return deps(types, t["Def"], prefix, seen, cur_api)
    elif t["Kind"] == "Array":
        return deps(types, t["Child"], prefix, seen, cur_api)
    elif t["Kind"] == "Enum":
        return set([(cur_api, f"{prefix}{t['Name']}")])
    elif t["Kind"] in ("Struct", "Union"):
        result = set()
        for type in t["NestedTypes"]:
            result |= deps(types, type, prefix + f"{t['Name']}::", seen, cur_api, True)
        for field in t["Fields"]:
            result |= deps(types, field["Type"], "", seen, cur_api)
        return result
    elif t["Kind"] == "FunctionPointer":
        result = set()
        for field in t["Params"]:
            result |= deps(types, field["Type"], prefix, seen, cur_api)
        result |= deps(types, t["ReturnType"], prefix, seen, cur_api)
        return result
    elif t["Kind"] == "Com":
        result = set()
        for method in t["Methods"]:
            for param in method["Params"]:
                result |= deps(types, param["Type"], prefix, seen, cur_api)
            result |= deps(types, method["ReturnType"], prefix, seen, cur_api)
        return result
    elif t["Kind"] in ("LPArray", "PointerTo"):
        return deps(types, t["Child"], prefix, seen, cur_api)
    else:
        return set()

types_needed = defaultdict(lambda: {})
for arch, func_info in libs.items():
    for lib_name, func_list in func_info.items():
        types_needed[arch][lib_name] = []
        seen = set()
        for api, func_dict in func_list.items():
            for func_name, func in func_dict.items():
                for param in func["Params"]:
                    types_needed[arch][lib_name].extend(deps(types, param["Type"], "", seen, api))
                types_needed[arch][lib_name].extend(deps(types, func["ReturnType"], "", seen, api))

needed_by = defaultdict(lambda: defaultdict(lambda: set()))
for arch in arch_list:
    for name, t in types_needed[arch].items():
        for api, type_name in t:
            needed_by[arch][(api, type_name)].add(name)

most_common_types = defaultdict(lambda: set())
dependencies = defaultdict(lambda: set())
for arch in arch_list:
    for (api, type_name), names in needed_by[arch].items():
        if len(names) > 1:
            most_common_types[arch].add((api, type_name))
            dependencies[arch] |= deps(types["X64"], types["X64"][api][type_name], "", set(), api)

for arch in arch_list:
    print(arch)
    print(f"  {sum([len(t) for t in types[arch].values()])} - Count of all types")
    print(f"  {len(most_common_types[arch])}  - Count of types which appear in more than one library")
    print(f"  {len(dependencies[arch])} - Dependencies of these common types")
    most_common_types[arch] |= dependencies[arch]
    print(f"  {len(most_common_types[arch])} - Dependencies and common types")

for arch, total in zip(arch_list, [33245, 33247, 33237]):
    cur_total = sum([len(type) for type in types[arch].values()])
    assert cur_total == total, f"Total {arch} types not equal to {total} instead {cur_total}"
print("Success")

NameError: name 'libs' is not defined

## Build the TypeLibraries

## TypeLibrary Checks

In [1]:
from typing import Dict
from binaryninja.architecture import Architecture
from binaryninja.platform import Platform
from binaryninja.typelibrary import TypeLibrary
from binaryninja.enums import StructureVariant, NamedTypeReferenceClass, TypeClass
from binaryninja.types import (Type, Tuple, PointerType, StructureType, EnumerationType, FunctionType, EnumerationMember, EnumerationBuilder, QualifiedName, FunctionParameter, PointerBuilder)
from binaryninja.log import log_to_stdout
from binaryninja.enums import LogLevel
from typing import Optional, Set, List, DefaultDict
from pathlib import Path
import json, codecs
from collections import defaultdict

lib_sets = {
    ("X64", "x86_64", Architecture["x86_64"], Platform["windows-x86_64"], "winX64common"),
    ("X86", "x86", Architecture["x86"], Platform["windows-x86"], "win32common"),
    ("Arm64", "aarch64", Architecture["aarch64"], Platform["windows-aarch64"], "winArm64common"),
}
def pointer_width(arch):
    return 4 if arch_name == "X86" else 8

def get_ntr_type(type_info) -> NamedTypeReferenceClass:
    if type_info["Kind"] == "Struct":
        return NamedTypeReferenceClass.StructNamedTypeClass
    elif type_info["Kind"] == "Union":
        return NamedTypeReferenceClass.UnionNamedTypeClass
    elif type_info["Kind"] == "Enum":
        return NamedTypeReferenceClass.EnumNamedTypeClass
    else:
        return NamedTypeReferenceClass.TypedefNamedTypeClass

def type_size(t, arch_name, nested_sizes={}) -> int:
    kind = t["Kind"]
    if kind == "Native":
        name = t["Name"]
        if name in ("Byte", "SByte", "Char"):
            return 1
        elif name in ("UInt16", "Int16"):
            return 2
        elif name in ("UInt32", "Int32", "Single", "Boolean"):
            return 4
        elif name in ("UInt64", "Int64", "Double"):
            return 8
        elif name in ("UIntPtr", "IntPtr"):
            return pointer_width(arch_name)
        elif name == "Void":
            assert False, "Attempting to get size of void"
            return 0
        elif name == "Guid":
            return 16
    elif kind == "ApiRef":
        name = t["Name"]
        if name in nested_sizes:
            return nested_sizes[name]
        type = types[arch_name][t["Api"]][name]
        assert type is not None, f"{arch_name, t['Api'], name} returned None"
        return type_size(type, arch_name, nested_sizes)
    elif kind == "NativeTypedef":
        return type_size(t["Def"], arch_name, nested_sizes)
    elif kind in ("PointerTo", "FunctionPointer", "LPArray"):
        return pointer_width(arch_name)
    elif kind == "Struct":
        new_nested_sizes = {}
        for nested_type in t["NestedTypes"]:
            new_nested_sizes[nested_type["Name"]] = type_size(nested_type, arch_name, nested_sizes)
        offset = 0
        for field in t["Fields"]:
            offset += type_size(field["Type"], arch_name, new_nested_sizes)
            if t["PackingSize"] != 0 and offset % t["PackingSize"] != 0:
                offset += t["PackingSize"] - (offset & (t["PackingSize"] - 1))
        return offset
    elif kind == "Union":
        new_nested_sizes = {}
        for nested_type in t["NestedTypes"]:
            new_nested_sizes[nested_type["Name"]] = type_size(nested_type, arch_name, nested_sizes)
        return max([type_size(field["Type"], arch_name, new_nested_sizes) for field in t["Fields"]])
    elif kind == "Enum":
        if t["IntegerBase"] is None:
            return 4
        x = {"Kind":"Native", "Name":t["IntegerBase"]}
        return type_size(x, arch_name, nested_sizes)
    elif kind == "Array":
        if t["Shape"]:
            return int(t["Shape"]["Size"]) * type_size(t["Child"], arch_name, nested_sizes)
        else:
            return pointer_width(arch_name)
    elif kind == "Com":
        return pointer_width(arch) * len(t["Methods"])
    else:
        assert False, f"Gettings size of {t['Kind']}"

defined_types:DefaultDict[str, DefaultDict[str, DefaultDict[str, Type]]] = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None)))
def add_to_type_library(typelib:TypeLibrary, arch_name:str, api_name:str, type_name:Optional[QualifiedName], t, seen:Dict[Tuple[str, str, str], Type], dependent:Optional[TypeLibrary] = None, nested_types = None) -> Optional[Type]:
    kind = t["Kind"]
    if kind == "Native":
        name = t["Name"]
        type = defined_types[arch_name][api_name][name]
        if type is not None:
            return type
        if name == "Byte":
            return Type.int(1, sign=False)
        elif name == "SByte":
            return Type.int(1)
        elif name == "Char":
            return Type.char()
        elif name == "UInt16":
            return Type.int(2, sign=False)
        elif name == "Int16":
            return Type.int(2)
        elif name == "Int64":
            return Type.int(8)
        elif name == "UInt32":
            return Type.int(4, sign=False)
        elif name == "UInt64":
            return Type.int(8, sign=False)
        elif name == "Int32":
            return Type.int(4)
        elif name == "Single":
            return Type.float(4)
        elif name == "Double":
            return Type.float(8)
        elif name == "UIntPtr":
            # TODO: Ensure integer width is correct
            return Type.pointer_of_width(pointer_width(arch_name), Type.int(pointer_width(arch_name), sign=False))
        elif name == "IntPtr":
            # TODO: Ensure integer width is correct
            return Type.pointer_of_width(pointer_width(arch_name), Type.int(pointer_width(arch_name), sign=True))
        elif name == "Void":
            return Type.void()
        elif name == "Boolean":
            return Type.bool()
        elif name == "Guid":
            new_type = Type.structure([Type.int(4, False), Type.int(2, False), Type.int(2, False), Type.array(Type.char(), 8)])
            assert new_type is not None
            typelib.add_named_type(QualifiedName("GUID"), new_type)
            return Type.void()
        else:
            assert False, f"Unhandled Native Type: {name}"
    elif kind == "ApiRef":
        name = t["Name"]
        api = t["Api"]
        if dependent is not None and dependent.get_named_type(name) is not None:
            result = dependent.get_named_type(name)
            dep_name = dependent.name
            assert dep_name is not None and result is not None

            if not isinstance(result, NamedTypeReferenceType):
                if isinstance(result, StructureType):
                    if result.union:
                        ntr_type = NamedTypeReferenceClass.UnionNamedTypeClass
                    else:
                        ntr_type = NamedTypeReferenceClass.StructNamedTypeClass
                elif isinstance(result, EnumerationType):
                    ntr_type = NamedTypeReferenceClass.EnumNamedTypeClass
                else:
                    ntr_type = NamedTypeReferenceClass.TypedefNamedTypeClass

                result = Type.named_type_reference(ntr_type, name, f"{api_name}::{name}")
                assert result is not None
            return result

        if nested_types is not None:
            for nested_type in nested_types:
                if name == nested_type["Name"]:
                    seen[(arch_name, api, name)] = None
                    result = add_to_type_library(typelib, arch_name, api_name, name, nested_type, seen, dependent, nested_types)
                    assert result is not None
                    return result

        if (arch_name, api, name) in seen:
            # we have a recursively defined type
            type = seen[(arch_name, api, name)]
            if type is not None:
                return type
            new_type_info = types[arch_name][api][name]
            assert new_type_info is not None
            result = Type.named_type_reference(get_ntr_type(new_type_info), name, f"{api}::{name}")
            assert result is not None
            seen[(arch_name, api, name)] = result
            return result

        new_type_info = types[arch_name][api][name]
        assert new_type_info is not None, f"{arch_name, api, name} returned None"
        seen[(arch_name, api, name)] = None
        new_type = add_to_type_library(typelib, arch_name, api, name, new_type_info, seen, dependent, nested_types)
        assert new_type is not None
        if not isinstance(new_type, NamedTypeReferenceType):
            # result = Type.named_type_reference(get_ntr_type(new_type_info), name, f"{api}::{name}")
            typelib.add_named_type(name, new_type)
            new_type = Type.named_type_reference(get_ntr_type(t), name, f"{api_name}::{name}")
            assert new_type is not None
        return new_type
    elif kind == "NativeTypedef":
        name = t["Name"]
        if dependent is not None and dependent.get_named_type(name) is not None:
            result = dependent.get_named_type(name)
            assert result is not None
            if not isinstance(result, NamedTypeReferenceType):
                new_type = Type.named_type_reference(get_ntr_type(t["Def"]), name, f"{api_name}::{name}")
                assert new_type is not None
            return new_type
        new_type = add_to_type_library(typelib, arch_name, api_name, None, t["Def"], seen, dependent, nested_types)
        seen[(arch_name, api_name, name)] = None
        assert new_type is not None
        typelib.add_named_type(name, new_type)
        new_type = Type.named_type_reference(get_ntr_type(t["Def"]), name, f"{api_name}::{name}")
        assert new_type is not None
        return new_type
    elif kind == "Array":
        child = t["Child"]
        if t["Shape"]:
            child = add_to_type_library(typelib, arch_name, api_name, None, t["Child"], seen, dependent, nested_types)
            assert child is not None
            return Type.array(child, int(t["Shape"]["Size"]))
        else:
            child = add_to_type_library(typelib, arch_name, api_name, None, t["Child"], seen, dependent, nested_types)
            assert child is not None
            return Type.pointer_of_width(pointer_width(arch_name), child)
    elif kind == "Enum":
        name = t["Name"]
        members = [EnumerationMember(str(member["Name"]), int(member["Value"])) for member in t["Values"]]
        result = EnumerationType.create(members, width=type_size(t, arch_name))
        assert result is not None
        seen[(arch_name, api_name, name)] = result
        typelib.add_named_type(name, result)
        new_type = Type.named_type_reference(get_ntr_type(t), name, f"{api_name}::{name}")
        assert new_type is not None
        return new_type
    elif kind in ("Struct", "Union"):
        name = t["Name"]
        members = []
        offset = 0
        seen[(arch_name, api_name, name)] = None
        nested_sizes = {}
        for nested_type in t["NestedTypes"]:
            nested_sizes[nested_type["Name"]] = type_size(nested_type, arch_name)
        for field in t["Fields"]:
            type = add_to_type_library(typelib, arch_name, api_name, None, field["Type"], seen, dependent, t["NestedTypes"])
            assert type is not None, f"returned none: {arch_name} {api_name}, {field['Type']}"
            members.append(StructureMember(type, None if field["Name"] == "Anonymous" else field["Name"] , offset))
            size = type_size(field['Type'], arch_name, nested_sizes)
            offset += size
            if t["PackingSize"] != 0 and offset % t["PackingSize"] != 0:
                offset += t["PackingSize"] - (offset & (t["PackingSize"] - 1))

        result = StructureType.create(members, True, StructureVariant.UnionStructureType if kind == "Union" else StructureVariant.StructStructureType)
        assert result is not None
        typelib.add_named_type(name, result)
        new_type = Type.named_type_reference(get_ntr_type(t), name, f"{api_name}::{name}")
        assert new_type is not None
        return new_type
    elif kind == "FunctionPointer":
        name = t["Name"]
        params = []
        for param in t["Params"]:
            type = add_to_type_library(typelib, arch_name, api_name, None, param["Type"], seen, dependent, nested_types)
            params.append((param["Name"], type))
        return_type = add_to_type_library(typelib, arch_name, api_name, None, t["ReturnType"], seen, dependent, nested_types)
        target = FunctionType.create(return_type, params)
        result = Type.pointer_of_width(pointer_width(arch_name), target)
        assert result is not None
        typelib.add_named_type(name, result)
        new_type = Type.named_type_reference(get_ntr_type(t), name, f"{api_name}::{name}")
        assert new_type is not None
        return new_type
    elif kind == "Com":
        return Type.void()
        # TODO: Probably need to actually define these
        # members = []
        # for method in t["Methods"]:
        #     params = []
        #     for param in method["Params"]:
        #         param_type = define_type(arch, api, param["Type"], needed, prefix)
        #         assert param_type is not None
        #         params.append(FunctionParameter(param_type, param["Name"]))
        #     return_type = define_type(arch, api, method["ReturnType"], needed, prefix)
        #     assert return_type is not None
        #     func = FunctionType.create(return_type, params)
        #     members.append((Type.pointer_of_width(pointer_width(arch), func), method["Name"]))
        # new_type = StructureType.create(members)
        # assert new_type is not None
        # return new_type
    elif kind in ("LPArray", "PointerTo"):
        new_type = add_to_type_library(typelib, arch_name, api_name, None, t["Child"], seen, dependent, nested_types)
        assert new_type is not None, t["Child"]
        result = Type.pointer_of_width(pointer_width(arch_name), new_type)
        assert result is not None
        return result
    elif kind == "ComClassID":
        return Type.void() # TODO: What is this really?
    elif kind == "MissingClrType":
        return Type.void()
    else:
        print(f"Found unknown type kind: {t['Kind']}")

win_common = {}
for key, arch_name, arch, platform, bntl_name in lib_sets:
    defined_types = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None)))
    win_common[key] = TypeLibrary.new(arch, bntl_name)
    win_common[key].add_platform(platform)
    filename = f"output/{arch_name}/{bntl_name}.bntl"
    print(len(most_common_types[key]), key, arch_name, platform, bntl_name, filename)

    for api, type_name in most_common_types[key]:
        type = types[key][api][type_name]
        if type is None:
            if not type_name.startswith("_"):
                print("couldn't find: ", key, api, type_name)
            continue
        t = add_to_type_library(win_common[key], key, api, QualifiedName(type_name), type, {})
        # if t is not None and win_common[key].get_named_type(type_name) is not None:
        #     win_common[key].add_named_type(QualifiedName(type_name), t)
    win_common[key].finalize()
    win_common[key].write_to_file(filename)

for arch, total in zip(arch_list, [575, 576, 574]):
    cur_total = len(win_common[arch].named_types)
    assert cur_total == total, f"Total for {arch} doesn't match {total} instead {cur_total}"
assert win_common["X64"].get_named_type("BSTR").type_class == TypeClass.PointerTypeClass
print("Success")


NameError: name 'most_common_types' is not defined

In [2]:
from typing import Set, List
import json
from tqdm.notebook import tqdm

ordinal_data = {}
ordinal_data["Arm64"] = {}
ordinal_data["X86"] = {}
ordinal_data["X64"] = json.load(open("mappingfile.json"))
typelibs = {}
for key, arch_name, arch, platform, dependency_name in tqdm(lib_sets, total=len(lib_sets), desc=f"Building Type Libraries"):
    for lib_name, func_list in tqdm(libs[key].items(), total=len(libs[key]), unit="typelibs", desc=f"Building {arch_name} Type Libraries"):
        if key != "X64":
            continue
        typelib = TypeLibrary.new(arch, f"{lib_name.lower()}.dll")
        typelib.add_platform(platform)
        typelib.dependency_name = dependency_name
        lookup_name = f"{lib_name.lower()}.dll.bntl"
        if lookup_name in ordinal_data[key]:
            typelib.guid = str(ordinal_data[key][lookup_name]["guid"])
            ordinal_name = ordinal_data[key][lookup_name]["ordinals"]
            if ordinal_name is not None:
                typelib.store_metadata("ordinals", ordinal_name)
                typelib.store_metadata(ordinal_name, ordinal_data[key][lookup_name][ordinal_name])
        for api_name, func_dict in func_list.items():
            for func_name, func in func_dict.items():
                params = []
                for param in func["Params"]:
                    param_type = add_to_type_library(typelib, key, api_name, None, param["Type"], {}, win_common[key])
                    params.append((param["Name"], param_type))
                return_type = add_to_type_library(typelib, key, api_name, None, func["ReturnType"], {}, win_common[key])
                function_type = Type.function(return_type, params)
                typelib.add_named_object(func_name, function_type)

        for name in win_common[key].named_types.keys():
            typelib.add_type_source(QualifiedName(name), win_common[key].name)

        typelib.finalize()
        typelib.write_to_file(f"output/{arch_name}/{lookup_name}")
        typelibs[f"output/{arch_name}/{lookup_name}"] = typelib

NameError: name 'lib_sets' is not defined

In [58]:
tl = win_common["X64"]
print(tl.get_named_type("HWND"))
tl = typelibs['output/x86_64/shell32.dll.bntl']
print(tl.get_named_type("SHELLEXECUTEINFOW").members)
print(hex(type_size(types["X64"]["UI.Shell"]["SHELLEXECUTEINFOW"], "X64", {})))

int64_t*
[<uint32_t cbSize, offset 0x0>, <uint32_t fMask, offset 0x4>, <HWND hwnd, offset 0x8>, <PWSTR lpVerb, offset 0x10>, <PWSTR lpFile, offset 0x18>, <PWSTR lpParameters, offset 0x20>, <PWSTR lpDirectory, offset 0x28>, <int32_t nShow, offset 0x30>, <HINSTANCE hInstApp, offset 0x34>, <void* lpIDList, offset 0x3c>, <PWSTR lpClass, offset 0x44>, <HKEY hkeyClass, offset 0x4c>, <uint32_t dwHotKey, offset 0x54>, <union _Anonymous_e__Union Anonymous, offset 0x58>, <HANDLE hProcess, offset 0x60>]
0x68


In [515]:

paths = list(Path("output/").glob("*/*.bntl"))
for file in tqdm(paths, total=len(paths), desc=f"validating TypeLibraries"):
    typelib = TypeLibrary.load_from_file(str(file))
    map_arch = {"aarch64":"Arm64", "x86":"X86", "x86_64":"X64"}
    lib_name = file.stem[:-4]
    # count up the number of named objects these files are supposed to have
    cur_total = sum([len(a.values()) for a in libs[map_arch[typelib.arch.name]][lib_name].values()])
    assert len(typelib.named_objects) == cur_total, f"total {cur_total} for {str(file)} isn't whats in the db"
print("Success")

validating TypeLibraries:   0%|          | 0/1018 [00:00<?, ?it/s]

Success


In [460]:
tl = TypeLibrary.load_from_file("/Users/peterlafosse/src/binaryninja/typelib/x86_64/libc.so.6.bntl")
# for n, f in tl.named_objects.items():
#     print(n, f)
tl.get_named_object("_IO_default_finish").parameters[0].type.target.type_class

<TypeClass.NamedTypeReferenceClass: 11>